# Demo: Serenity Derivatives API - Yield Curve and Vol Surface

Serenity builds in sophisticated option and rates analytics as part of its core offering, and these functions
are all exposed via the API. This notebook shows how you can use it to fetch yield curves and volatility
surfaces.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
from os import getenv
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(getenv('SERENITY_CONFIG_ID', None))

In [ ]:
from datetime import datetime, timedelta
from uuid import UUID, uuid4
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from serenity_sdk.renderers.derivatives.reference_data import get_predefined_option_infos

from serenity_sdk.renderers.derivatives.widget_tools import YieldCurveVersionTimeChooser, VolatilitySurfaceVersionTimeChooser
from serenity_sdk.renderers.derivatives.table_plot import (
    YieldCurveTablePlot, 
    VolatilitySurfaceTablePlot, 
    plot_volatility_surface_3d
)

# plot parameters
plt.rcParams['font.size'] = '16'

# create an alias to the api
api = connect_widget.get_api()

# Set time range

In [ ]:
# Pick the time range to view
# To get the most recent ones, set both timestamps to None
# start_dt, end_dt = None, None
# To browse different versions over time, set the start and end timestamps, accordingly
# e.g. uncomment this
start_dt, end_dt = (datetime.utcnow() - timedelta(days=2)), datetime.utcnow()

# Yield Curves

## Pick Version
There are multiple type of market data built on multiple datetimes (e.g. hourly, daily). 
Pick the version to show and plot. 

In [ ]:
vtc_yc = YieldCurveVersionTimeChooser(api=api, start_datetime=start_dt, end_datetime=end_dt)
print('Pick the curve and as-of-time from the drop-down menus')
display(vtc_yc.get_widget_to_display())

## Now, load the selected version

In [ ]:
# load the version through API
id_selected, as_of_time_selected = vtc_yc.get_id_as_of_time()
yc_selected = api.pricer().get_yield_curve_version(id_selected, as_of_time_selected)
print(f'Loaded id={id_selected}, as-of-time={as_of_time_selected}')

# use a helper object to show the content
yc_tp = YieldCurveTablePlot(yc_selected)

### Peeking data

In [ ]:
# peeking raw data
yc_tp.raw_pts.head(2)

In [ ]:
# peeking interpolated data
yc_tp.interpolated_curve.head(2)

## Plotting
* Top: future prices and corresponding index (underlying) spot price. 
* Middle: The flat-forward interpolation method is used to get the interpolated points.
* Bottom: It is equivalent to the log-linear interpolation in the discount factor.

In [ ]:
print('There are small variations in mark price spot across expiries - probably due to timing of the market data collection')
yc_tp.plot()
plt.show()

# Volatility Surfaces

## Pick Version
There are multiple type of market data built on multiple datetimes (e.g. hourly, daily). 
Pick the version to show and plot. 

In [ ]:
vtc_vol =  VolatilitySurfaceVersionTimeChooser(api, start_datetime=start_dt, end_datetime=end_dt)

print('Pick the vol surface and build time from the drop-down menus')
vtc_vol.widget_name.value = vtc_vol.widget_name.options[1] # choose the log_money version
display(vtc_vol.get_widget_to_display())

## Now, load the selected version

In [ ]:
# load the version through API
id_selected, as_of_time_selected = vtc_vol.get_id_as_of_time()
vs_selected = api.pricer().get_volatility_surface_version(id_selected, as_of_time_selected)
print(f'Loaded id={id_selected}, as-of-time={as_of_time_selected}')


# use a helper object to show the content 
vs_tp = VolatilitySurfaceTablePlot(vs_selected)

### Peeking data

In [ ]:
# showing available expiries
vs_tp.time_to_expiries

In [ ]:
# peeking raw data
vs_tp.raw_pts.head(2)

In [ ]:
# If you want to see 'display_name', we need to merge with the supported (predefined) option info.

# Note: We may not find 'display_name's for some of the short-dated options if they have expired by now.
# This will be improved in due course.  
predefined_option_infos = get_predefined_option_infos(api)
vs_raw_pts_with_display_name = pd.merge(vs_tp.raw_pts, predefined_option_infos[['asset_id', 'display_name']], 
         how='left', left_on = 'option_asset_id', right_on = 'asset_id').drop('asset_id', axis=1)
vs_raw_pts_with_display_name

In [ ]:
# peeking at interpolated data
vs_tp.interpolated_surface.head(2)

In [ ]:
# show calibration SVI parameters
df = pd.DataFrame(vs_tp.vs.interpolated.calibration_params).T
df.index.name = 'time-to-expiry'
df

## Plotting
For each expiries, show a smile curve with input data (dots)

In [ ]:
vs_tp.plot()

Plot at a fixed log moneyness

In [ ]:
log_moneyness = 0.0 # try -0.05, 0.05 as well
time_to_expiries, vols = vs_tp.plot_at_log_moneyness(log_moneyness)

# 3D interactive plot

In [ ]:
fig = plot_volatility_surface_3d(vs_selected)
fig.show()

# END